## Extracting Title and Authors from scientific articles

In [1]:
import pathlib, fitz
import requests
import random
### pdftitle quickly extracts the title from the pdf metadata

paper_number = random.randint(0,18)
print(paper_number)

# We extract the full text
fname = 'examples/paper_'+str(paper_number)+'.pdf'  # get document filename
with fitz.open(fname) as doc:  # open document
    text = "".join([page.get_text() for page in doc])
# write as a binary file to support non-ASCII characters
pathlib.Path(fname + ".txt").write_bytes(text.encode())


# We split the text by page to be used as input to OpenAI
# My idea is to get 10 questions per page
text_by_page = []
with fitz.open(fname) as doc:
  for page in doc:
    text_by_page.append(page.get_text())   


6


In [2]:
# All scientific articles have a DOI
# All DOIs start with 10
def get_doi():
  doi_index = text.find('doi')
  if doi_index == -1:
    doi_index = text.find('DOI')
  if doi_index == -1:
    doi_index == text.find('Doi')  

  doi_start = text.find('10', doi_index)  
  end_space = text.find(' ', doi_start)
  end_line = text.find('\n', doi_start)
  if  end_space < end_line:
    doi_end = end_space
  else:
    doi_end = end_line  
  DOI = text[doi_start:doi_end]

  # If the DOI is cut we do a second search:
  if len(DOI)<10:
    doi_index2 = text.find('doi', doi_index+1)
    doi_start = text.find('10', doi_index2)  
    end_space = text.find(' ', doi_start)
    end_line = text.find('\n', doi_start)
    if  end_space < end_line:
      doi_end = end_space
    else:
      doi_end = end_line  
  DOI = text[doi_start:doi_end]   
  return DOI 

In [3]:
def get_biblio_data(DOI):
  base_url = f"https://doi.org/{DOI}"
  headers = {"Accept": "text/bibliography; style=bibtex"}
  response = requests.get(base_url, headers=headers)

  if response.status_code == 200:
    bibtex = response.text.strip()
  else:
    print(response.status_code)
    print(response.text)
    return None

  title_init = bibtex.find('title')
  title_end = bibtex.find('}', title_init)
  author_init = bibtex.find('author')
  author_end = bibtex.find('}', author_init)
  
  title = bibtex[title_init:title_end+1]
  authors = bibtex[author_init:author_end+1]
  return [title, authors]

DOI = get_doi()
print(DOI)
get_biblio_data(DOI)

10.1088/1475-7516/2012/02/029


['title={Dark matter as the trigger of strong electroweak phase transition}',
 'author={Chowdhury, Talal Ahmed and NemevÅ¡ek, Miha and SenjanoviÄ\x87, Goran and Zhang, Yue}']